In [1]:
import math
import warnings
import numpy as np

In [2]:
def f(A, B, V, t):
    PQx = V.x - (A.x + ((B.x - A.x) * t))
    PQy = V.y - (A.y + ((B.y - A.y) * t))
    PQz = 0
    PQ = [PQx, PQy, PQz]
    AB = [
        B.x - A.x,
        B.y - A.y,
        B.z - A.z
    ]
    magnitudePQ = math.sqrt(PQx ** 2.0 + PQy ** 2.0 + PQz ** 2.0)
    try:
        forceMagnitude = 1 / magnitudePQ
    except ZeroDivisionError:
        return 999999
    forceDirection = np.cross(PQ, [0, 0, 1]) / magnitudePQ
    forceVector = forceMagnitude * forceDirection
    return np.dot(forceVector, AB)


def integral(A, B, V):
    """Do the line integral"""
    numberofRectangles = 1000
    startingt = 0.0
    endingt = 1.0
    width = (endingt - startingt) / numberofRectangles
    runningSum = 0.0
    for i in range(numberofRectangles):
        height = f(A, B, V, startingt + i * width)
        area = height * width
        runningSum += area
    return runningSum


class Vertex(object):
    """The three dimensional coordinates of a vertex"""
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return str([self.x, self.y, self.z])

    def is_blocked(self, faces):
        """True when this vertex is blocked from view by faces"""
        for face in faces.values():
            if face.is_blocking(self):
                return True
        return False

    @staticmethod
    def load(filename):
        # vertices (vid, x, y, z)
        np_vertices = np.loadtxt(filename, delimiter="\t", skiprows=1,
                                 dtype=[('id', 'i4'), ('x', 'f8'), ('y', 'f8'), ('z', 'f8')])
        vertices = {}
        for id, x, y, z in np_vertices:
            vertices[id] = Vertex(x, y, z)
        return vertices


class Edge(object):
    """The vertex ID's of two vertices"""
    def __init__(self, v1, v2):
        self.v1 = v1
        self.v2 = v2

    def __repr__(self):
        return str([self.v1, self.v2])

    @staticmethod
    def load(filename, vertices):
        # edge vertices (eid, vid, vid)
        np_edges = np.loadtxt(filename, delimiter="\t", skiprows=1, dtype="i4,i4,i4")

        edges = {}
        for id, v1, v2 in np_edges:
            edges[id] = Edge(vertices[v1], vertices[v2])
        return edges


class Face(object):
    ZERO_EQUIVALENCE = 0.00000001

    def __init__(self, e1, e2, e3, e4):
        self.e1 = e1
        self.e2 = e2
        self.e3 = e3
        self.e4 = e4

    def __str__(self):
        return str([self.e1, self.e2, self.e3, self.e4])

    def is_blocking(self, vertex):
        """True if the vertex is blocked by this face

        Accumulate the integral of each edge in this face with the vertex. If the total is
        zero, then the vertex is visible.
        """
        sum = 0.0
        for edge in [self.e1, self.e2, self.e3, self.e4]:
            if edge is not None:
                sum += integral(edge.v1, edge.v2, vertex)
        return abs(sum) < Face.ZERO_EQUIVALENCE

    @staticmethod
    def load(filename, edges):
        # face edges (fid, eid, eid, eid, eid)
        # if the last edge is null indicate with a -1 edge id
        convertfunc = lambda x: -1 if x == b'NULL' else x
        np_faceedges = np.loadtxt(filename, delimiter="\t", skiprows=1,
                                  converters={4: convertfunc},
                                  dtype=[('id', 'i4'), ('edge1', 'i4'), ('edge2', 'i4'), ('edge3', 'i4'), ('edge4', 'i4')])

        faces = {}
        for id, eid1, eid2, eid3, eid4 in np_faceedges:
            try:
                if eid4 == -1:
                    args = edges[eid1], edges[eid2], edges[eid3], None
                else:
                    args = edges[eid1], edges[eid2], edges[eid3], edges[eid4]
                faces[id] = Face(*args)
            except KeyError:
                #warnings.warn("Face %s discarded because not all these edges are known (%s, %s, %s, %s)." % (id, eid1, eid2, eid3, eid4))
                pass
        return faces

In [3]:
vertices = Vertex.load('../headmesh/NVertices.txt')
edges = Edge.load('../headmesh/NEdgeVertices.txt', vertices)
faces = Face.load('../headmesh/NFaceEdges.txt', edges)
print("Loaded %s vertices, %s edges, %s faces." % (len(vertices), len(edges), len(faces)))

Loaded 775 vertices, 1520 edges, 752 faces.


In [6]:
print("Vertex: ", vertices[0])
print("Edge: ", edges[0])
print("Face: ", faces[0])

Vertex:  [1.4822599889999999, -9.4975204469999994, 8.6801795960000003]
Edge:  [[1.4822599889999999, -9.4975204469999994, 8.6801795960000003], [1.045789957, -9.7227897639999998, 6.7623200419999998]]
Face:  [[[1.4822599889999999, -9.4975204469999994, 8.6801795960000003], [1.045789957, -9.7227897639999998, 6.7623200419999998]], [[2.3399600980000002, -9.1642999649999997, 8.2300500870000004], [1.4822599889999999, -9.4975204469999994, 8.6801795960000003]], [[1.045789957, -9.7227897639999998, 6.7623200419999998], [2.3399600980000002, -9.1642999649999997, 8.2300500870000004]], None]


In [12]:
face = faces[0]
vertex = vertices[400]
print("Vertex")
print(vertex)
print("Face")
print(face)
print("This face is blocking this vertex?")
print(face.is_blocking(vertex))

Vertex
[2.8961300849999998, 9.9518299100000007, 8.2101297379999991]
Face
[[[1.4822599889999999, -9.4975204469999994, 8.6801795960000003], [1.045789957, -9.7227897639999998, 6.7623200419999998]], [[2.3399600980000002, -9.1642999649999997, 8.2300500870000004], [1.4822599889999999, -9.4975204469999994, 8.6801795960000003]], [[1.045789957, -9.7227897639999998, 6.7623200419999998], [2.3399600980000002, -9.1642999649999997, 8.2300500870000004]], None]
Is blocking?
False


In [16]:
vertex = vertices[400]
print("Vertex", vertex)
print("There are %s Faces" % len(faces))
print("Is this vertex blocked by any one of those faces?")
print(face.is_blocking(vertex))

Vertex [2.8961300849999998, 9.9518299100000007, 8.2101297379999991]
There are 752 Faces
Is this vertex blocked by any one of those faces?
False
